# **<span style="color: black;">STAT 201 Project Proposal - Group 38</span>** 
##### **-- by Pushya Jain, Jingyuan Liu, Ty Stevenson & Bowen Tian**

## **Statistical inference on crime statistics of Vancouver**

### **Introduction**

<span style="color: blue;">In our STAT201 group project, we propose the following question of interest: How does the mean number of crimes per hour vary between violent and non-violent crimes throughout the day? With the rise of unprovoked stranger attacks and safety concerns for Vancouverites, we will seek to better understand the relation of criminal activity and time of day. The data is provided by the Vancouver Police Department on the open source “geodash” website. We will be using the data from 2003 to 2022 for all neighborhoods within the Vancouver municipal proper. The goal of this study is to understand if daytime and nighttime hours tend to harbour different types of crimes, which may be useful in further inferences about the motivations behind such crimes (CS Wisdom et H. Torch). These trends can also assist in predicting criminal activity and developing preventive and punitive methods to reduce crime, and understand the underlying social problems conducive to crimes (University of Minnesota).</span>

### **Preliminary results**

In [1]:
### Load all necessary packages into R and set the seed. Run this cell before continuing.
library(tidyverse)
library(tidymodels)
library(gridExtra)
library(repr)
library(cowplot)
library(broom)
library(digest)
library(infer)
library(gridExtra)
set.seed(38)
options(repr.matrix.max.rows = 8)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.1     ✔ rsample      1.1.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.3     ✔ workflows    1.0.0
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.1     ✔ yardstick    1.1.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() ma

In [2]:
#Read the raw data from the web using its url.
url <- "https://raw.githubusercontent.com/PUSH-YA/stat-201-2022w1-group-38/main/crimedata_csv_AllNeighbourhoods_AllYears.csv"
vancouver_crime_raw_data <- read_csv(url)
head(vancouver_crime_raw_data)

Rows: 842492 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2017,2,21,22,0,1XX W GEORGIA ST,Central Business District,491892.3,5458408
Theft from Vehicle,2017,3,8,12,45,1XX W GEORGIA ST,Central Business District,491892.3,5458408
Theft from Vehicle,2017,3,14,20,0,1XX W GEORGIA ST,Central Business District,491892.3,5458408
Theft from Vehicle,2017,5,8,9,47,1XX W GEORGIA ST,Central Business District,491892.3,5458408
Theft from Vehicle,2017,5,17,13,18,1XX W GEORGIA ST,Central Business District,491892.3,5458408
Theft from Vehicle,2017,6,16,17,30,1XX W GEORGIA ST,Central Business District,491892.3,5458408


- Read the data set from the web 
- clean the data
- Plot some graphs answering some things about the question
- Compute estimates of the parameter you identified across your groups. Present this in a table. If relevant, include these estimates in your plot.

### **Methods: plan**

- In what ways is this report trustworthy
- Analyse the plot and explain what it means
- Explain why just plots are not enough to give a *stakeholder* [not sure what it means]
- What should you provide to address the gap
- What should you expect your results to be
- State the ***null hypothesis*** and the ***alternative hypothesis***
- What impact could such findings have?
- What future questions could this lead to?

### **References**

References on:
- VPD OPEN DATA. (n.d.). Retrieved from https://geodash.vpd.ca/opendata/
-  Chester Ismay and Albert Y. Kim (2022e, October 28). Statistical Inference via Data Science. https://moderndive.com/
- information about crime in vancouver
- what crime statistics are valuable